In [1]:
%load_ext autoreload

%autoreload 2

In [1]:
import numpy as np
import GPy
import altair as alt
from typing import List
from numpy import ndarray
from src.observations import Observations
from src.data_packer import DataPacker
from src.generator import OneDimensionalGenerator
from src.generator import function_proxy
from src.generator import function_field
from src.generator import function_predict
from src.coregionalized import Coregionalized
from src.coregionalization_input import CoregionalizationInput
from src.plotter import Plotter

plotter = Plotter()

n_feats = 1
n_obs_field = 2
n_obs_proxy = 5
n_obs_predict = 10
spread = 1

range_predict = [0, 2]
range_proxy = [0, 2]
range_field = [0, 1]

g1 = OneDimensionalGenerator(f=function_predict, task_index=0)
X1 = np.random.uniform(low=range_predict[0], 
                       high=range_predict[1],
                       size=(n_obs_predict, n_feats))
predict_observations = g1.generate(X1)

g2 = OneDimensionalGenerator(f=function_proxy, task_index=1)
X2 = np.random.uniform(low=range_proxy[0], 
                       high=range_proxy[1],
                       size=(n_obs_proxy, n_feats))
proxy_observations = g2.generate(X2)

g3 = OneDimensionalGenerator(f=function_field, task_index=2)
X3 = np.random.uniform(low=range_field[0], 
                       high=range_field[1],
                       size=(n_obs_field, n_feats))
field_observations = g3.generate(X3)

packer = DataPacker()
cr_input: CoregionalizationInput = packer.pack([predict_observations, proxy_observations, field_observations])

coregionalized = Coregionalized(num_tasks=3, num_feats=n_feats)
coregionalized.fit(cr_input.X, cr_input.Y, cr_input.task_indexes)

def convert_tasks_to_str(_tasks):
    mapper = {0: "Predict", 1: "Proxy", 2: "Field"}
    
    _tasks = _tasks.ravel().tolist()
    return [mapper[i] for i in _tasks]

points = plotter.points_plot(cr_input.X, cr_input.Y, 
                             cr_input.task_indexes,
                             legend=None)

X, m, v, tasks = coregionalized.predict_region(region_start=0)

lines = plotter.line_plot(X, m, tasks, legend=None)
b = plotter.bands_plot(X, Y=m, tasks=tasks, variance=v, legend=None)

alt.layer(lines, points, b).resolve_scale(shape='independent', color='independent')

 /opt/homebrew/anaconda3/envs/causaldsr/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.


SchemaValidationError: Invalid specification

        altair.vegalite.v4.schema.channels.Shape->type, validating 'enum'

        'quantitative' is not one of ['nominal', 'ordinal', 'geojson']
        

alt.LayerChart(...)